# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import time
from vaastav.fbref import *

# Getting raw data

In [3]:
def get_url( url ):
    df = pd.read_html( url, header=1)[0]
    return df

In [4]:
def get_logs_for_players_in_season(players, season, players_names=[]):
    logs = pd.DataFrame({'A' : []})
    for id, player in list(players.items()):
        player_name = player.data[0]['player'].replace(" ", "-").replace("ï", "i").replace("é", "e").replace("á", "a").replace("ó", "o").replace("Á", "A").replace("ø", "o").replace("ć", "c").replace("í", "i").replace("ú", "u").replace("Ć", "C").replace("ã", "a").replace("ğ", "g").replace("ş", "s").replace("É", "E").replace("ñ", "n").replace("Ł", "L").replace("ń", "n").replace("ß", "ss").replace("ç", "c").replace("İ", "I").replace("č", "c").replace("ö", "o").replace("ë", "e").replace("š", "s").replace("ä", "a").replace("Ç", "C").replace("ü", "u").replace("Ø", "O").replace("ú", "u").replace("ú", "u")
        if players_names and player_name not in players_names: 
            continue
        print("Getting data for " + player_name)
        url = 'https://fbref.com/en/players/' + str(id) + '/matchlogs/' + season + '/summary/' + player_name + '-Match-Logs'
        new_player_df = get_url(url)
        new_player_df["Name"] = player_name
        
        # dropping NaN rows
        new_player_df = new_player_df[new_player_df['Date'].notna()]
        
        # only Premier League
        new_player_df = new_player_df[new_player_df.Comp == "Premier League"]
        
        # only last 15 matches
        new_player_df = new_player_df[-15:]
        
        if not logs.empty:
            logs = pd.concat([logs, new_player_df])
        else:
            logs = new_player_df
        
        time.sleep(3.5)
    return logs

In [5]:
players, stats = get_epl_players()

In [6]:
%%time
# player_names = ["Mohamed-Salah", "Miguel-Almiron", "Bukayo-Saka", "Kevin-De-Bruyne", "Wilfried-Zaha", "Mason-Mount", "Bruno-Fernandes"]
logs = get_logs_for_players_in_season(players, '2022-2023', None)

Getting data for Brenden-Aaronson
Getting data for Che-Adams
Getting data for Tyler-Adams
Getting data for Tosin-Adarabioyo
Getting data for Nayef-Aguerd
Getting data for Rayan-Ait-Nouri
Getting data for Kristoffer-Ajer
Getting data for Manuel-Akanji
Getting data for Nathan-Ake
Getting data for Marc-Albrighton
Getting data for Thiago-Alcantara
Getting data for Trent-Alexander-Arnold
Getting data for Alisson
Getting data for Dele-Alli
Getting data for Miguel-Almiron
Getting data for Julian-Alvarez
Getting data for Daniel-Amartey
Getting data for Joachim-Andersen
Getting data for Elliot-Anderson
Getting data for Jaidon-Anthony
Getting data for Michail-Antonio
Getting data for Antony
Getting data for Cameron-Archer
Getting data for Alphonse-Areola
Getting data for Joe-Aribo
Getting data for Adam-Armstrong
Getting data for Stuart-Armstrong
Getting data for Kepa-Arrizabalaga
Getting data for Pierre-Emerick-Aubameyang
Getting data for Ludwig-Augustinsson
Getting data for Serge-Aurier
Getting

# Modyfing data

In [7]:
# splitting result column
logs[['WDL', 'GoalsTeams']] = logs['Result'].str.split(' ', expand=True)
logs[['Team Score', 'Opp Score']] = logs['GoalsTeams'].str.split('–', expand=True)
logs['Team Score'] = logs['Team Score'].astype(int)
logs['Opp Score'] = logs['Opp Score'].astype(int)
logs = logs.drop(['Result'], axis=1)

In [9]:
logs.replace("On matchday squad, but did not play", 0, inplace=True)
logs["Min"] = logs["Min"].astype(float)
logs["Gls"] = logs["Gls"].astype(float)
logs["Ast"] = logs["Ast"].astype(float)

In [10]:
# additional columns
# approximated FPL points
logs["CS"] = (logs["Opp Score"] == 0).astype(float)
logs["Was Home"] = (logs["Venue"] == "Home").astype(float)
logs["GW"] = logs['Round'].str.split(' ', expand=True)[1]
logs["FPL"] = 1 + (logs["Min"] >= 60) + (5 * logs["Gls"]) + (3 * logs["Ast"]) + (1 * logs["CS"])

In [11]:
logs

,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,Pos,Min,...,Match Report,Name,WDL,GoalsTeams,Team Score,Opp Score,CS,Was Home,GW,FPL
2,2022-08-21,Sun,Premier League,Matchweek 3,Home,Leeds United,Chelsea,Y,AM,82.0,...,Match Report,Brenden-Aaronson,W,3–0,3,0,1.0,1.0,3,8.0
4,2022-08-27,Sat,Premier League,Matchweek 4,Away,Leeds United,Brighton,Y,"AM,RW",81.0,...,Match Report,Brenden-Aaronson,L,0–1,0,1,0.0,0.0,4,2.0
5,2022-08-30,Tue,Premier League,Matchweek 5,Home,Leeds United,Everton,Y,"AM,RW",75.0,...,Match Report,Brenden-Aaronson,D,1–1,1,1,0.0,1.0,5,5.0
6,2022-09-03,Sat,Premier League,Matchweek 6,Away,Leeds United,Brentford,Y,AM,90.0,...,Match Report,Brenden-Aaronson,L,2–5,2,5,0.0,0.0,6,2.0
9,2022-10-02,Sun,Premier League,Matchweek 9,Home,Leeds United,Aston Villa,Y,"AM,CM",78.0,...,Match Report,Brenden-Aaronson,D,0–0,0,0,1.0,1.0,9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,2022-11-06,Sun,Premier League,Matchweek 15,Away,Arsenal,Chelsea,Y*,RM,86.0,...,Match Report,Martin-Odegaard,W,1–0,1,0,1.0,0.0,15,3.0
25,2022-11-12,Sat,Premier League,Matchweek 16,Away,Arsenal,Wolves,Y*,RM,90.0,...,Match Report,Martin-Odegaard,W,2–0,2,0,1.0,0.0,16,13.0
28,2022-12-26,Mon,Premier League,Matchweek 17,Home,Arsenal,West Ham,Y*,RM,90.0,...,Match Report,Martin-Odegaard,W,3–1,3,1,0.0,1.0,17,8.0
29,2022-12-31,Sat,Premier League,Matchweek 18,Away,Arsenal,Brighton,Y*,RM,86.0,...,Match Report,Martin-Odegaard,W,4–2,4,2,0.0,0.0,18,10.0


# Saving to csv

In [12]:
logs.to_csv("data/logs_all.csv")

# Features

In [13]:
info = ["Date", "Day", "GW", "Was Home"]
features = ["Min", "Gls", "Sh", "SoT", "xG", "npxG", "xAG", "CS"]
to_predict = ["FPL"]

In [14]:
logs[info + features + to_predict]

,Date,Day,GW,Was Home,Min,Gls,Sh,SoT,xG,npxG,xAG,CS,FPL
2,2022-08-21,Sun,3,1.0,82.0,1.0,2,2,1.0,1.0,0.1,1.0,8.0
4,2022-08-27,Sat,4,0.0,81.0,0.0,0,0,0.0,0.0,0.0,0.0,2.0
5,2022-08-30,Tue,5,1.0,75.0,0.0,1,1,0.1,0.1,0.1,0.0,5.0
6,2022-09-03,Sat,6,0.0,90.0,0.0,1,0,0.0,0.0,0.3,0.0,2.0
9,2022-10-02,Sun,9,1.0,78.0,0.0,0,0,0.0,0.0,0.1,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,2022-11-06,Sun,15,0.0,86.0,0.0,1,0,0.1,0.1,0.2,1.0,3.0
25,2022-11-12,Sat,16,0.0,90.0,2.0,3,2,1.5,1.5,0.0,1.0,13.0
28,2022-12-26,Mon,17,1.0,90.0,0.0,6,0,0.4,0.4,1.1,0.0,8.0
29,2022-12-31,Sat,18,0.0,86.0,1.0,1,1,0.1,0.1,0.8,0.0,10.0


In [15]:
logs[info + features + to_predict].to_csv("data/logs_features.csv")